In [1]:
#!/usr/bin/env python3
"""
CiNii Articleのjsonを連結するに
"""

import urllib3
import urllib.parse
import certifi
import pandas as pd
import csv
import sys
import json
import time

#APPID = "xxxxx" # APPIDはCiNiiで取得してセット
APPID = "CiNiixxxxxxxxxxxxxxxxxxxxxxxxxxxxx"
keyword2 = "大向一輝"
keyword = urllib.parse.quote(keyword2)
count = 50

req_url = 'https://ci.nii.ac.jp/books/opensearch/search?format=json'
req_url = req_url + "&appid=" + APPID
http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED', ca_certs=certifi.where())

def req(keyword, start):
   req_url2 = req_url + "&q=" + keyword + "&start=" + str(start) + "&count=" + str(count)
   print(req_url2)
   response = http.request('GET', req_url2)
   json_data = response.data.decode('utf-8')
   time.sleep(0.5)
   return json_data

data_dict1 = json.loads(req(keyword,1))
totalResults = data_dict1["@graph"][0]["opensearch:totalResults"] # 総件数
print("totalResults: ",totalResults)

data_dict = pd.DataFrame()

num = 1
while True:
    json_data2 = req(keyword,num)
    try:
        data_dict = data_dict.append( pd.read_json(json.dumps(json.loads(json_data2)["@graph"][0]["items"])) ,sort="true")
    except Exception:
        pass
    # 
    num = num + count
    if num > int(totalResults) :
        break



https://ci.nii.ac.jp/books/opensearch/search?format=json&appid=CiNii09-89ef94917d5946d2a0dae832519f760b&q=%E5%A4%A7%E5%90%91%E4%B8%80%E8%BC%9D&start=1&count=50
totalResults:  8
https://ci.nii.ac.jp/books/opensearch/search?format=json&appid=CiNii09-89ef94917d5946d2a0dae832519f760b&q=%E5%A4%A7%E5%90%91%E4%B8%80%E8%BC%9D&start=1&count=50


In [2]:
data_dict.tail()

,@id,@type,cinii:ownerCount,dc:creator,dc:date,dc:publisher,dcterms:hasPart,dcterms:isPartOf,link,prism:publicationDate,rdfs:seeAlso,title
3,http://ci.nii.ac.jp/ncid/BB11534438,item,104,"トム・ヒース, クリスチャン・バイツァー著 ; 武田英明 [ほか] 訳",2013,[近代科学社],[{'@id': 'urn:isbn:9784764904279'}],NaN,{'@id': 'http://ci.nii.ac.jp/ncid/BB11534438'},2013,{'@id': 'http://ci.nii.ac.jp/ncid/BB11534438.j...,Linked data : webをグローバルなデータ空間にする仕組み
4,http://ci.nii.ac.jp/ncid/BB08796640,item,193,"大向一輝, 池谷瑠絵著",2012,[丸善出版],[{'@id': 'urn:isbn:9784621053812'}],[{'@id': 'http://ci.nii.ac.jp/ncid/BN06190301'...,{'@id': 'http://ci.nii.ac.jp/ncid/BB08796640'},2012,{'@id': 'http://ci.nii.ac.jp/ncid/BB08796640.j...,ウェブらしさを考える本 : つながり社会のゆくえ
5,http://ci.nii.ac.jp/ncid/BB18528058,item,249,高野明彦監修,2015,[KADOKAWA],[{'@id': 'urn:isbn:9784046538888'}],[{'@id': 'http://ci.nii.ac.jp/ncid/BB17029642'...,{'@id': 'http://ci.nii.ac.jp/ncid/BB18528058'},2015,{'@id': 'http://ci.nii.ac.jp/ncid/BB18528058.j...,検索の新地平 : 集める、探す、見つける、眺める
6,http://ci.nii.ac.jp/ncid/BB08818776,item,1,NaN,2012,[全国共同利用・共同研究拠点「人文学諸領域の複合的共同研究国際拠点」],NaN,NaN,{'@id': 'http://ci.nii.ac.jp/ncid/BB08818776'},2012,{'@id': 'http://ci.nii.ac.jp/ncid/BB08818776.j...,情報の構造とメタデータ
7,http://ci.nii.ac.jp/ncid/BB13061654,item,182,"楊暁捷, 小松和彦, 荒木浩編",2013,[勉誠出版],[{'@id': 'urn:isbn:9784585200239'}],NaN,{'@id': 'http://ci.nii.ac.jp/ncid/BB13061654'},2013,{'@id': 'http://ci.nii.ac.jp/ncid/BB13061654.j...,デジタル人文学のすすめ


In [3]:
# DataFrameの列@idからlistを作成
l_id = data_dict['@id'].values.tolist()
# l_id  # ->['http://ci.nii.ac.jp/ncid/BN07817320', 'http://ci.nii.ac.jp/ncid/BN05475634',,,,,,

In [4]:
# １件ごとの json を出力
def req(id):
    req_url = id + ".json"
    # print(req_url)
    response = http.request('GET', req_url)
    json_data= response.data.decode('utf-8')
    # json_data = json_data2.pop('bibo:owner')
    time.sleep(0.2)
    return json_data

data_dictj = {}
data_dictj2 = {}
# print(type(data_dictj))

for id in l_id:
    print(type(data_dictj), id)
    # idをキーにして、連結
    data_dictj[id] = json.loads(req(id))
    # 所蔵館と'@context'のデータ要素を削除
    del data_dictj[id]['@graph'][0]['bibo:owner'],data_dictj[id]['@context']

<class 'dict'> http://ci.nii.ac.jp/ncid/BA8164687X
<class 'dict'> http://ci.nii.ac.jp/ncid/BB02488158
<class 'dict'> http://ci.nii.ac.jp/ncid/BB18501492
<class 'dict'> http://ci.nii.ac.jp/ncid/BB11534438
<class 'dict'> http://ci.nii.ac.jp/ncid/BB08796640
<class 'dict'> http://ci.nii.ac.jp/ncid/BB18528058
<class 'dict'> http://ci.nii.ac.jp/ncid/BB08818776
<class 'dict'> http://ci.nii.ac.jp/ncid/BB13061654


In [5]:
# ensure_ascii オプションがTrueだと非ASCII文字を全て "\uXXXX" の形にエスケープ
# with open('test2.json', 'w', encoding='utf_8_sig') as f:
with open('test2.json', 'w', encoding='utf_8') as f:
    json.dump(data_dictj, f, indent=4, ensure_ascii=False)
# with open('test2.json', encoding='unicode-escape') as f:    
with open('test2.json', encoding='utf-8') as f:
    print(f.read())

{
    "http://ci.nii.ac.jp/ncid/BA8164687X": {
        "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json",
        "@graph": [
            {
                "@id": "http://ci.nii.ac.jp/ncid/BA8164687X#entity",
                "@type": "bibo:Book",
                "foaf:isPrimaryTopicOf": {
                    "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json"
                },
                "dc:title": [
                    {
                        "@value": "ウェブがわかる本"
                    },
                    {
                        "@value": "ウェブ ガ ワカル ホン",
                        "@language": "ja-hrkt"
                    }
                ],
                "dc:creator": "大向一輝著",
                "dc:publisher": [
                    "岩波書店"
                ],
                "dcterms:extent": "ix, 179p",
                "cinii:size": "18cm",
                "dc:language": "jpn",
                "dc:date": "2007",
                "cinii:ncid": "BA8164687X",
                "cini

In [6]:
# ensure_ascii オプションがTrueだと非ASCII文字を全て "\uXXXX" の形にエスケープ
# with open('test2.json', 'w', encoding='utf_8_sig') as f:
with open('test2.json', 'w', encoding='utf_8') as f:
    json.dump(data_dictj, f, indent=4, ensure_ascii=False)
# with open('test2.json', encoding='unicode-escape') as f:    
with open('test2.json', encoding='utf-8') as f:
    print(f.read())

{
    "http://ci.nii.ac.jp/ncid/BA8164687X": {
        "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json",
        "@graph": [
            {
                "@id": "http://ci.nii.ac.jp/ncid/BA8164687X#entity",
                "@type": "bibo:Book",
                "foaf:isPrimaryTopicOf": {
                    "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json"
                },
                "dc:title": [
                    {
                        "@value": "ウェブがわかる本"
                    },
                    {
                        "@value": "ウェブ ガ ワカル ホン",
                        "@language": "ja-hrkt"
                    }
                ],
                "dc:creator": "大向一輝著",
                "dc:publisher": [
                    "岩波書店"
                ],
                "dcterms:extent": "ix, 179p",
                "cinii:size": "18cm",
                "dc:language": "jpn",
                "dc:date": "2007",
                "cinii:ncid": "BA8164687X",
                "cini

In [7]:
# ensure_ascii オプションがTrueだと非ASCII文字を全て "\uXXXX" の形にエスケープ
# with open('test2.json', 'w', encoding='utf_8_sig') as f:
with open('test2.json', 'w', encoding='utf_8') as f:
    json.dump(data_dictj, f, indent=4, ensure_ascii=False)
# with open('test2.json', encoding='unicode-escape') as f:    
with open('test2.json', encoding='utf-8') as f:
    print(f.read())

{
    "http://ci.nii.ac.jp/ncid/BA8164687X": {
        "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json",
        "@graph": [
            {
                "@id": "http://ci.nii.ac.jp/ncid/BA8164687X#entity",
                "@type": "bibo:Book",
                "foaf:isPrimaryTopicOf": {
                    "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json"
                },
                "dc:title": [
                    {
                        "@value": "ウェブがわかる本"
                    },
                    {
                        "@value": "ウェブ ガ ワカル ホン",
                        "@language": "ja-hrkt"
                    }
                ],
                "dc:creator": "大向一輝著",
                "dc:publisher": [
                    "岩波書店"
                ],
                "dcterms:extent": "ix, 179p",
                "cinii:size": "18cm",
                "dc:language": "jpn",
                "dc:date": "2007",
                "cinii:ncid": "BA8164687X",
                "cini

In [8]:
print(data_dictj)
# print(data_dictj["@graph"][0]) # 

{'http://ci.nii.ac.jp/ncid/BA8164687X': {'@id': 'http://ci.nii.ac.jp/ncid/BA8164687X.json', '@graph': [{'@id': 'http://ci.nii.ac.jp/ncid/BA8164687X#entity', '@type': 'bibo:Book', 'foaf:isPrimaryTopicOf': {'@id': 'http://ci.nii.ac.jp/ncid/BA8164687X.json'}, 'dc:title': [{'@value': 'ウェブがわかる本'}, {'@value': 'ウェブ ガ ワカル ホン', '@language': 'ja-hrkt'}], 'dc:creator': '大向一輝著', 'dc:publisher': ['岩波書店'], 'dcterms:extent': 'ix, 179p', 'cinii:size': '18cm', 'dc:language': 'jpn', 'dc:date': '2007', 'cinii:ncid': 'BA8164687X', 'cinii:ownerCount': '165', 'foaf:maker': [{'@id': 'http://ci.nii.ac.jp/author/DA15839119#entity', '@type': 'foaf:Person', 'foaf:name': [{'@value': '大向, 一輝'}, {'@value': 'オオムカイ, イッキ', '@language': 'ja-hrkt'}]}], 'prism:publicationDate': ['2007.4'], 'foaf:topic': [{'@id': 'http://ci.nii.ac.jp/books/search?q=WWW', 'dc:title': 'WWW'}], 'dcterms:isPartOf': [{'@id': 'http://ci.nii.ac.jp/ncid/BN00972289#entity', 'dc:title': '岩波ジュニア新書, 562', '@type': 'bibo:Book'}], 'dcterms:hasPart': [{

In [9]:
# ensure_ascii オプションがTrueだと非ASCII文字を全て "\uXXXX" の形にエスケープ
# with open('test2.json', 'w', encoding='utf_8_sig') as f:
with open('test2.json', 'w', encoding='utf_8') as f:
    json.dump(data_dictj, f, indent=4, ensure_ascii=False)
# with open('test2.json', encoding='unicode-escape') as f:    
with open('test2.json', encoding='utf-8') as f:
    print(f.read())

{
    "http://ci.nii.ac.jp/ncid/BA8164687X": {
        "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json",
        "@graph": [
            {
                "@id": "http://ci.nii.ac.jp/ncid/BA8164687X#entity",
                "@type": "bibo:Book",
                "foaf:isPrimaryTopicOf": {
                    "@id": "http://ci.nii.ac.jp/ncid/BA8164687X.json"
                },
                "dc:title": [
                    {
                        "@value": "ウェブがわかる本"
                    },
                    {
                        "@value": "ウェブ ガ ワカル ホン",
                        "@language": "ja-hrkt"
                    }
                ],
                "dc:creator": "大向一輝著",
                "dc:publisher": [
                    "岩波書店"
                ],
                "dcterms:extent": "ix, 179p",
                "cinii:size": "18cm",
                "dc:language": "jpn",
                "dc:date": "2007",
                "cinii:ncid": "BA8164687X",
                "cini

In [28]:
# dict 型のキー一覧の先頭を取得する next(iter(data_dictj))
graph_keys = data_dictj[next(iter(data_dictj))]['@graph'][0].keys()
list(dict.fromkeys(graph_keys))
print(graph_keys)

dict_keys(['@id', '@type', 'foaf:isPrimaryTopicOf', 'dc:title', 'dc:creator', 'dc:publisher', 'dcterms:extent', 'cinii:size', 'dc:language', 'dc:date', 'cinii:ncid', 'cinii:ownerCount', 'foaf:maker', 'prism:publicationDate', 'foaf:topic', 'dcterms:isPartOf', 'dcterms:hasPart'])


In [30]:
graph_keys = []
# dict 型のキー一覧の先頭を取得する next(iter(data_dictj))
# 全データからキーを取得して追記する。
for k in data_dictj:
    graph_keys.extend(data_dictj[k]['@graph'][0].keys())
    # print(graph_keys)
# 順序を維持してユニークなキーを取得    
graph_keys = list(dict.fromkeys(graph_keys))
print(graph_keys)    

['@id', '@type', 'foaf:isPrimaryTopicOf', 'dc:title', 'dc:creator', 'dc:publisher', 'dcterms:extent', 'cinii:size', 'dc:language', 'dc:date', 'cinii:ncid', 'cinii:ownerCount', 'foaf:maker', 'prism:publicationDate', 'foaf:topic', 'dcterms:isPartOf', 'dcterms:hasPart', 'dcterms:alternative', 'cinii:note', 'dc:subject', 'cinii:contentOfWorks']


In [31]:
import csv
with open('test2.csv', 'w', encoding='utf_8') as f:
    writer = csv.DictWriter(f, graph_keys)
    writer.writeheader()
    # writer.writerows(data_dictj[next(iter(data_dictj))]['@graph'])
    for k in data_dictj:
        # writer.writeheader()
        writer.writerows(data_dictj[k]['@graph'])
# with open('test2.json', encoding='unicode-escape') as f:    
with open('test2.csv', encoding='utf-8') as f:
    print(f.read())

@id,@type,foaf:isPrimaryTopicOf,dc:title,dc:creator,dc:publisher,dcterms:extent,cinii:size,dc:language,dc:date,cinii:ncid,cinii:ownerCount,foaf:maker,prism:publicationDate,foaf:topic,dcterms:isPartOf,dcterms:hasPart,dcterms:alternative,cinii:note,dc:subject,cinii:contentOfWorks

http://ci.nii.ac.jp/ncid/BA8164687X#entity,bibo:Book,{'@id': 'http://ci.nii.ac.jp/ncid/BA8164687X.json'},"[{'@value': 'ウェブがわかる本'}, {'@value': 'ウェブ ガ ワカル ホン', '@language': 'ja-hrkt'}]",大向一輝著,['岩波書店'],"ix, 179p",18cm,jpn,2007,BA8164687X,165,"[{'@id': 'http://ci.nii.ac.jp/author/DA15839119#entity', '@type': 'foaf:Person', 'foaf:name': [{'@value': '大向, 一輝'}, {'@value': 'オオムカイ, イッキ', '@language': 'ja-hrkt'}]}]",['2007.4'],"[{'@id': 'http://ci.nii.ac.jp/books/search?q=WWW', 'dc:title': 'WWW'}]","[{'@id': 'http://ci.nii.ac.jp/ncid/BN00972289#entity', 'dc:title': '岩波ジュニア新書, 562', '@type': 'bibo:Book'}]",[{'@id': 'urn:isbn:9784005005628'}],,,,

http://ci.nii.ac.jp/ncid/BB02488158#entity,bibo:Book,{'@id': 'http://ci.nii.

In [15]:
for k in data_dictj:
    print(k)

http://ci.nii.ac.jp/ncid/BA8164687X
http://ci.nii.ac.jp/ncid/BB02488158
http://ci.nii.ac.jp/ncid/BB18501492
http://ci.nii.ac.jp/ncid/BB11534438
http://ci.nii.ac.jp/ncid/BB08796640
http://ci.nii.ac.jp/ncid/BB18528058
http://ci.nii.ac.jp/ncid/BB08818776
http://ci.nii.ac.jp/ncid/BB13061654
